# Prompting examples

In [3]:
import cohere
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()
api_key = os.getenv("COHERE_API_KEY")

In [5]:
co = cohere.ClientV2()

## Example n° 1: Entity extraction

We will use the LLM to extract the following entities from the medical text.
- Patient:
  - Name
  - Age
- Admission date
- Symptoms
- Diagnosis
- Recommended treatment

The output will be a string in JSON format, which will be converted into a Python dictionary.

In [6]:
text_to_analyze = """Jane Doe, a 34-year-old woman, was admitted to City General Hospital on May 15, 2024, due to a high fever and persistent cough./ 
After several tests, she was diagnosed with bronchitis. The attending physician, Dr. Michael Green, prescribed a course of antibiotics and rest./ 
The next evaluation is scheduled for May 22."""

In [7]:
system_prompt = """You are an expert in medical data extraction. 
Only extract relevant information from the text. 
If you do not know the value of an attribute, return an empty string."""

In [8]:
json_sample_format = {
  "patient": [{
    "name": "string",
    "age": "integer"
  }],
  "admission_date": "string",
  "symptoms": [
    "string"
  ],
  "diagnosis": "string",
  "treatment": [
    "string"
  ]
}


prompt = f""" Extract the following attributes from the text

    ### 
    Attributes to extract:
    - name = patient's name
    - age = patient's age
    - admission_date = date the patient is admitted to the medical facility
    - symptoms = list of symptoms the patient presents
    - diagnosis = diagnosis given to the patient
    - treatment = treatment to be carried out

    ###
    Text:
    {text_to_analyze}
    
    ###
    Remember, the output must be in JSON format with the following structure:

    {json.dumps(json_sample_format, indent=2)}
"""


In [9]:
response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "system", "content": system_prompt},
             {"role": "user", "content": prompt}],
    )

print(response.message.content[0].text)

{
  "patient": [
    {
      "name": "Jane Doe",
      "age": "34"
    }
  ],
  "admission_date": "May 15, 2024",
  "symptoms": [
    "high fever",
    "persistent cough"
  ],
  "diagnosis": "bronchitis",
  "treatment": [
    "antibiotics",
    "rest"
  ]
}


In [10]:
final_result = (response.message.content[0].text)
print(final_result)

{
  "patient": [
    {
      "name": "Jane Doe",
      "age": "34"
    }
  ],
  "admission_date": "May 15, 2024",
  "symptoms": [
    "high fever",
    "persistent cough"
  ],
  "diagnosis": "bronchitis",
  "treatment": [
    "antibiotics",
    "rest"
  ]
}


## Example n° 2: Function calling

In [11]:
contacts = {
'Pedro Sander':{'tel': 15456663258, 'mail': 'PedritoElSander@gmail.com'},
'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [12]:
def add_contact(name, phone, email):
    """
    Adds a contact to the dictionary.
    Parameters:
        name (str): Contact's name.
        phone (str): Contact's phone number.
        email (str): Contact's email address.
    Returns:
        str: Message confirming the addition of the contact.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return {'Response': "Contact added successfully."}

def get_information(name):
    """
    Retrieves contact information.
    Parameters:
        name (str): Contact's name.
    Returns:
        dict/str: Contact information or a message if the contact does not exist.
    """
    if name in contacts:
        return contacts[name]
    else:
        return {'Response': "Contact not found."}


In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Adds a contact to the directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Contact's name."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Contact's phone number."
                    },
                    "email": {
                        "type": "string",
                        "description": "Contact's email address."
                    }
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Retrieves contact information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Contact's name."
                    }
                },
                "required": ["name"]
            }
        }
    }
]

In [14]:
functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}

In [15]:
system_prompt = """You are an assistant specialized in managing a contact directory and answering related queries.

Your capabilities include:
- Adding new contacts with their name, phone number, and email
- Retrieving contact information
- Answering questions about stored contacts
- Using search tools when necessary to verify information

Contact information must include:
- Name (required)
- Phone number (required)
- Email address (required)
- Additional notes (optional)

You will maintain data privacy and only share contact information with authorized users."""

In [16]:
user_input = input("Ingrese su consulta")
print(user_input)

Who is Marta?


In [17]:
messages=[{"role": "system", "content": system_prompt},
          {"role": "user", "content": user_input}]

response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

In [18]:
if response.message.tool_calls:
    try:
        print("Function call detected: Adding tool calls and plan to request")
        messages.append({
            'role': 'assistant',
            'tool_calls': response.message.tool_calls,
            'tool_plan': response.message.tool_plan
        })
    except Exception as e:
        print(f"Error appending message: {e}")
else:
    try:
        print("Direct response: Outputting model's message\n")
        print(response.message.content[0].text)
    except IndexError:
        print("Error: No content available in response")

Function call detected: Adding tool calls and plan to request


In [19]:
tool_content_v2 = []
# Iterate over the tool calls generated by the model
for tc in response.message.tool_calls:
    # here is where you would call the tool recommended by the model, using the parameters recommended by the model
    tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # store the output in a list
    #tool_content.append(json.dumps(tool_result))
    
    for data in tool_result.items():
        tool_content_v2.append({"type": "document", "document": {"data": json.dumps(data)}})
        # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
   
    # append the chat history
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": tool_content_v2}
    )

In [20]:
response = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools
)

print(response.message.content[0].text)

I'm sorry, I can't find a contact by the name of Marta.


In [21]:
contacts

{'Pedro Sander': {'tel': 15456663258, 'mail': 'PedritoElSander@gmail.com'},
 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}}

In [22]:
def process_contact_query(user_input, model="command-r-plus-08-2024", cohere_client=None):
    """
    Process a user query related to contact management using Cohere's LLM.
    
    Args:
        user_input (str): The user's query or request
        model (str): The Cohere model to use
        cohere_client: The initialized Cohere client
        
    Returns:
        dict: Contains the response information and status
    """
    try:
        # Validate inputs
        if not cohere_client:
            raise ValueError("Cohere client is required")
        if not user_input.strip():
            raise ValueError("User input cannot be empty")

        # Define tools and functions
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "add_contact",
                    "description": "Adds a contact to the directory.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Contact's name."
                            },
                            "phone": {
                                "type": "string",
                                "description": "Contact's phone number."
                            },
                            "email": {
                                "type": "string",
                                "description": "Contact's email address."
                            }
                        },
                        "required": ["name", "phone", "email"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_information",
                    "description": "Retrieves contact information.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Contact's name."
                            }
                        },
                        "required": ["name"]
                    }
                }
            }
        ]

        functions_map = {
            "add_contact": add_contact,
            "get_information": get_information
        }

        system_prompt = """You are an assistant specialized in managing a contact directory and answering related queries.
        Your capabilities include:
        - Adding new contacts with their name, phone number, and email
        - Retrieving contact information
        - Answering questions about stored contacts
        - Using search tools when necessary to verify information
        Contact information must include:
        - Name (required)
        - Phone number (required)
        - Email address (required)
        - Additional notes (optional)
        You will maintain data privacy and only share contact information with authorized users."""

        # Initialize messages
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]

        # Get initial response from Cohere
        response = cohere_client.chat(
            model=model,
            messages=messages,
            tools=tools
        )

        # Process the response
        if response.message.tool_calls:
            messages.append({
                'role': 'assistant',
                'tool_calls': response.message.tool_calls,
                'tool_plan': response.message.tool_plan
            })

            # Process tool calls
            tool_results = []
            for tool_call in response.message.tool_calls:
                # Execute the function
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                tool_result = functions_map[function_name](**function_args)

                # Format the result
                tool_content = []
                for data in tool_result.items():
                    tool_content.append({
                        "type": "document",
                        "document": {"data": json.dumps(data)}
                    })

                # Store results
                tool_results.append({
                    "tool_call_id": tool_call.id,
                    "content": tool_content
                })
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_content
                })

            # Get final response from Cohere with tool results
            final_response = cohere_client.chat(
                model=model,
                messages=messages
            )

            return {
                "status": "success",
                "type": "tool_response",
                "message": final_response.message.content[0].text,
                "tool_results": tool_results
            }
        else:
            return {
                "status": "success",
                "type": "direct_response",
                "message": response.message.content[0].text
            }

    except Exception as e:
        return {
            "status": "error",
            "message": str(e),
            "type": "error"
        }

In [23]:
user_input = "Tell me Flavio Oncativo email."
result = process_contact_query(user_input, cohere_client=co)

if result["status"] == "success":
    print(result["message"])
    if result["type"] == "tool_response":
        print("Tool Results:", result["tool_results"])
else:
    print("Error:", result["message"])

Flavio Oncativo's email is FOncativo@hotmail.com.
Tool Results: [{'tool_call_id': 'get_information_br48nqn1aknq', 'content': [{'type': 'document', 'document': {'data': '["tel", 1545554178]'}}, {'type': 'document', 'document': {'data': '["mail", "FOncativo@hotmail.com"]'}}]}]


## Example n° 3: Story-Based Q&A

The "story_answer" function takes a question about a given context as input and outputs a response generated by an LLM. The function allows users to ask questions about a story and receives responses following these specific requirements:
Response Specifications:

- Answers must be based strictly on the provided story
- Must provide consistent answers for identical questions
- Responses limited to one sentence
- Response language matches the question's language (English, Spanish, or Portuguese)
- Must include relevant emojis that summarize the response content
- All responses written in third person
- Default response "Sorry, I cannot help you with that" for off-topic questions
 -Every response must end with "Hakuna Matata!" regardless of language or token count

In [24]:
story = """In the ancient world of Greek mythology, the hero Perseus was known for his courage and determination. His most famous adventure began when King Polydectes of Seriphos sent him on an impossible quest: to slay the Gorgon Medusa and bring back her head.

Medusa was once a beautiful maiden, but she had been cursed by the goddess Athena and transformed into a monstrous creature with snakes for hair. Anyone who gazed directly into her eyes would be turned to stone. Armed with gifts from the gods—Hermes' winged sandals, Hades' helm of invisibility, Athena's polished shield, and a sharp, curved sword—Perseus set off on his perilous journey.

Guided by the gods and the nymphs who provided him with vital tools, Perseus eventually found Medusa in her lair. Using Athena's shield as a mirror to avoid direct eye contact, he managed to approach the sleeping Gorgon. With one swift and precise strike, he decapitated Medusa, safely placing her head in a special bag to avoid its deadly gaze.

On his way back, Perseus encountered many adventures, including rescuing the princess Andromeda from a sea monster. He used Medusa's head to turn the monster to stone, saving Andromeda and earning her love.

Upon returning to Seriphos, Perseus presented Medusa's head to King Polydectes, who had been tormenting Perseus' mother. With a single glance, Medusa's gaze turned the tyrannical king and his followers to stone, freeing his mother from their grip.

Perseus went on to have many more adventures, but the tale of his victory over Medusa remained his most legendary feat, celebrated in the annals of Greek mythology. Through bravery, divine assistance, and quick thinking, he became one of the greatest heroes in ancient tales."""

In [25]:
def story_answer(answer):

    cohere_client = cohere.ClientV2()
    model = "command-r-plus-08-2024"
    
    system_prompt = f"""
    You are a storyteller, and you have been tasked with answering questions about a precise story.
    You have access to a specific story and you should answer question about it precisely.
    You have no access to anything else but that story.
    You should response using one sentence only.
    You should response using the same language as the answer was written.
    You are to respond to the questions in the third person. 
    You should add two emojis to sum up the sentence.
    If the question has no relation to the story context, the answer should be 'I'm sorry I can't help you with that'.
    Add 'Hakuna Matata!' always at the end of the sentence.

    ## The story
    {story}
    """

    messages = [ 
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": answer} 
        ]

    # Get initial response from Cohere
    response = cohere_client.chat(
        model=model,
        messages=messages
        )
    
    # Add to messages
    

    return response.message.content[0].text

In [26]:
story_answer("Who was Medusa?")

"Medusa was a cursed maiden, transformed into a Gorgon with the power to turn people to stone, and she was the target of Perseus' quest. 🧜\u200d♀️🐉 Hakuna Matata!"

## Example n° 4: Creating a Simple LLM-Powered Chatbot

This chatbot is designed for end users and generates responses with these specific characteristics:

- Keeps all responses under 70 tokens
- Maintains a positive and enthusiastic tone
- Provides helpful advice in a tutor-like manner

In [28]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

chat_history = [{"role": "system", "content":"Responde con tono entusiasta y da consejos utiles."}]

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    chat_history.append({"role": "user", "content": message})
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=chat_history,
    )

    # actualizo el historial
    text_response = response.message.content[0].text
    chat_history.append({"role": "assistant", "content": text_response})

    return response.message.content[0].text

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()